# Required Libraries

In [ ]:
# install.packages("gridExtra")

1. Preprocessing
    
    i. Remove Outliares

    ii. Normalization
    
2. Define Number of Clusters

    i. Manual Method

    ii. Automated Methods

3. k-means Clustering

    i. k=2

    ii. k=3

    iii. k=4
    
4. PCA

6. Clustering

In [ ]:
# libraries
library("readxl")
options(warn = 0)
library(dplyr)
library(ggplot2)
library(caTools)
library(caret)
library(GGally)
library(janitor)
library(corrplot)
library(tidyverse)
library(hrbrthemes)
library(viridis)
library(NbClust)
library(factoextra)
library(gridExtra)

# Loading Dataset 

In [ ]:
df <- read_excel("data/Whitewine_v2.xlsx")
#add index column to data frame
df$index <- 1:nrow(df)
head(df)

replacing each quality measures into 1-4 range

1 wrost and 4 is best

In [ ]:
df["quality"][df["quality"] == 5] <- 1
df["quality"][df["quality"] == 6] <- 2
df["quality"][df["quality"] == 7] <- 3
df["quality"][df["quality"] == 8] <- 4

In [ ]:
head(df)

In [ ]:
boxplot(df[1:11])

In [ ]:
# format columns
df <- janitor::clean_names(df)

# EDA 

Getting some idea about the dataset

In [ ]:
summary(df)

with the minimum and maximum value of the each column I was able to get an idea about which column may be have some outliars, 

residual sugar column's minimum is 0.600 and maximum is 65.800, as the givan description about features there are some rare wine categories in this dataframe.

PH scale also should be 3<PH<4 and there may be outliares in PH column.

we have to deal with outliares in next steps

In [ ]:
#check null values
sum(is.na(df))

there are no null values in this dataset

In [ ]:
#Correlation Heatmap of Variables
corrplot(cor(df[1:12]))

as heatmap suggestions alcohol jas strongest correlation with the wine quality

# Outlier Remove

using data vizializations to detect outliaers

### IQR

In [ ]:
z_score = function(x) {
  return((x - mean(x)) / sd(x))
}

In [ ]:
#find Q1, Q3, and interquartile range for values in column A
Q1 <- quantile(df$total_sulfur_dioxide, .25)
Q3 <- quantile(df$total_sulfur_dioxide, .75)
IQR <- IQR(df$total_sulfur_dioxide)

#only keep rows in dataframe that have values within 1.5*IQR of Q1 and Q3
no_outliers_IQR <- subset(df, df$total_sulfur_dioxide> (Q1 - 1.5*IQR) & df$total_sulfur_dioxide< (Q3 + 1.5*IQR))

#view row and column count of new data frame
dim(no_outliers_IQR)

In [ ]:
# boxplot(no_outliers_IQR$total_sulfur_dioxide)

### Z score

In [ ]:
boxplot(df[1:11])

In [ ]:
z_score = function(x) {
  return((x - mean(x)) / sd(x))
}

In [ ]:
dfNorm <- as.data.frame(lapply(df[1:11], z_score))
dfNorm$index <- 1:nrow(dfNorm)
head(dfNorm)

In [ ]:
summary(dfNorm[1:11])

In [ ]:
#only keep rows in dataframe with all z-scores less than absolute value of 3 
no_outliers <- dfNorm[!rowSums(dfNorm[1:11]>3),]

In [ ]:
head(no_outliers)

In [ ]:
# merge normalized one ane orginal quality
# merge two data frames by ID
df_final <- merge(no_outliers,df[12:13],by="index")

In [ ]:
head(df_final)

In [ ]:
summary(df_final[2:12])

In [ ]:
histogram = function(x,title,x_label) {
  # Add a Normal Curve (Thanks to Peter Dalgaard)
  x <- x
  h<-hist(x, breaks=10, col="red", xlab=x_label,
    main=title)
  xfit<-seq(min(x),max(x),length=40)
  yfit<-dnorm(xfit,mean=mean(x),sd=sd(x))
  yfit <- yfit*diff(h$mids[1:2])*length(x)
  lines(xfit, yfit, col="blue", lwd=2)
}

In [ ]:
# histogram(df$fixed_acidity,"fixed_acidity values before preprocessing","fixed_acidity value")
# histogram(df_final$fixed_acidity,"fixed_acidity values after preprocessing","fixed_acidity value")
histogram(df$quality,"quality values before preprocessing","fixed_acidity value")
histogram(df_final$quality,"quality values after preprocessing","fixed_acidity value")

Define the number of cluster centres (via manual & automated tools). The
automated tools should include NBclust, Elbow and one from Gap statistics or silhouette methods

## Malual Methods - Define Cluster Centers

In [ ]:
head(df_final)

# Automatic Methods

#### NbClust

In [ ]:
# see the datasrt types
str(df_final)

In [ ]:
df_nb <- df_final[2:12]

In [ ]:
#setting seed point
set.seed(26)

In [ ]:
# #euclidient distance
# no_of_clusters_eud = NbClust(df_nb,distance="euclidean", min.nc=2,max.nc=10,method="kmeans",index="all")

In [ ]:
# #manhatten
# no_of_clusters_man = NbClust(df_nb,distance="manhattan", min.nc=2,max.nc=15,method="kmeans",index="all")

In [ ]:
# #maximum
# no_of_clusters_maximum = NbClust(df_nb,distance="maximum", min.nc=2,max.nc=15,method="kmeans",index="all")

#### Elbow

In [ ]:
fviz_nbclust(df_final[2:12],kmeans,method="wss")+geom_vline(xintercept=4,linetype=1)

In [ ]:
k = 1:10

In [ ]:

set.seed(25)	
WSS = sapply(k, function(k) {kmeans(df_final[2:12], centers=k)$tot.withinss})


In [ ]:
# You can then use a line plot to plot the within sum of squares with a different number of k
plot(k, WSS, type="b", xlab= "Number of k", ylab="Within sum of squares")


#### Silhoutte

In [ ]:
fviz_nbclust(df_final[2:12], kmeans, method = "silhouette") + 
labs(subtitle = "Silhoutte Method")

# k-means clustering

In [ ]:
head(df_final)

#### k=2

In [ ]:
x=df_final[2:12]
y=df_final$quality

In [ ]:
kc_2 <- kmeans(x,centers=2)

In [ ]:
kc_2

In [ ]:
str(kc_2)

In [ ]:
table(y,kc_2$cluster)

In [ ]:
# implement the metrix
confusionMatrix(

 factor(kc_2$cluster, levels = 1:4),

 factor(df_final$quality,levels=1:4)
)

In [ ]:
fviz_cluster(kc_2,data=x)

### k=3

In [ ]:
kc_3 <- kmeans(x,centers=3)

In [ ]:
kc_3

In [ ]:
str(kc_3)

In [ ]:
table(y,kc_3$cluster)

In [ ]:
# implement the metrix
confusionMatrix(

 factor(kc_3$cluster, levels = 1:4),

 factor(df_final$quality,levels=1:4)
)

In [ ]:
fviz_cluster(kc_3,data=x)

## k=4

In [ ]:
kc_4 <- kmeans(x,centers=4)

In [ ]:
kc_4

In [ ]:
str(kc_4)

In [ ]:
table(y,kc_4$cluster)

In [ ]:
# implement the metrix
confusionMatrix(

 factor(kc_4$cluster, levels = 1:4),

 factor(df_final$quality,levels=1:4)
)

In [ ]:
fviz_cluster(kc_4,data=x)

In [ ]:
# plots to compare
p1 <- fviz_cluster(kc_2, geom = "point", data = x) + ggtitle("k = 2")
p2 <- fviz_cluster(kc_3, geom = "point",  data = x) + ggtitle("k = 3")
p3 <- fviz_cluster(kc_4, geom = "point",  data = x) + ggtitle("k = 4")


grid.arrange(p1, p2, p3,nrow = 2)

In [ ]:
# distance <- get_dist(x)
# fviz_dist(distance, gradient = list(low = "#00AFBB", mid = "white", high = "#FC4E07"))

## PCA

In [ ]:
head(df_final)

In [ ]:
principle_components <- prcomp(df_final[2:12])

In [ ]:
principle_components

In [ ]:
summary(principle_components)

In [ ]:
plot(principle_components)

In [ ]:
plot(principle_components, type = "l")

In [ ]:
biplot(principle_components)

In [ ]:
biplot(principle_components, scale = 0)

In [ ]:
str(principle_components)

In [ ]:
df_final_with_pcs <- cbind(df_final, principle_components$x)
head(df_final_with_pcs)

In [ ]:
library(ggplot2)

ggplot(df_final_with_pcs, aes(PC9,PC10,PC11,col = quality, fill = quality)) +
  stat_ellipse(geom = "polygon", col = "black", alpha = 0.5) +
  geom_point(shape = 21, col = "black")

#### clustering

In [ ]:
x_2 = df_final_with_pcs[22:24]
y_2 = df_final_with_pcs$quality

In [ ]:
kcpca <- kmeans(x_2,4)

In [ ]:
kcpca

In [ ]:
fviz_cluster(kcpca,data=x_2)

In [ ]:
table(y_2,kcpca$cluster)

In [ ]:
# implement the metrix
confusionMatrix(

 factor(kcpca$cluster, levels = 1:4),

 factor(df_final$quality,levels=1:4)
)